In [1]:
import pandas as pd
from vertexai.evaluation import EvalTask
from vertexai.generative_models import GenerativeModel, HarmBlockThreshold, HarmCategory
from vertexai.preview.evaluation import notebook_utils


instruction = "Summarize the following article"

context = [
    "To make a classic spaghetti carbonara, start by bringing a large pot of salted water to a boil. While the water is heating up, cook pancetta or guanciale in a skillet with olive oil over medium heat until it's crispy and golden brown. Once the pancetta is done, remove it from the skillet and set it aside. In the same skillet, whisk together eggs, grated Parmesan cheese, and black pepper to make the sauce. When the pasta is cooked al dente, drain it and immediately toss it in the skillet with the egg mixture, adding a splash of the pasta cooking water to create a creamy sauce.",
    "Preparing a perfect risotto requires patience and attention to detail. Begin by heating butter in a large, heavy-bottomed pot over medium heat. Add finely chopped onions and minced garlic to the pot, and cook until they're soft and translucent, about 5 minutes. Next, add Arborio rice to the pot and cook, stirring constantly, until the grains are coated with the butter and begin to toast slightly. Pour in a splash of white wine and cook until it's absorbed. From there, gradually add hot chicken or vegetable broth to the rice, stirring frequently, until the risotto is creamy and the rice is tender with a slight bite.",
    "For a flavorful grilled steak, start by choosing a well-marbled cut of beef like ribeye or New York strip. Season the steak generously with kosher salt and freshly ground black pepper on both sides, pressing the seasoning into the meat. Preheat a grill to high heat and brush the grates with oil to prevent sticking. Place the seasoned steak on the grill and cook for about 4-5 minutes on each side for medium-rare, or adjust the cooking time to your desired level of doneness. Let the steak rest for a few minutes before slicing against the grain and serving.",
    "Creating a creamy homemade tomato soup is a comforting and simple process. Begin by heating olive oil in a large pot over medium heat. Add diced onions and minced garlic to the pot and cook until they're soft and fragrant. Next, add chopped fresh tomatoes, chicken or vegetable broth, and a sprig of fresh basil to the pot. Simmer the soup for about 20-30 minutes, or until the tomatoes are tender and falling apart. Remove the basil sprig and use an immersion blender to puree the soup until smooth. Season with salt and pepper to taste before serving.",
    "To bake a decadent chocolate cake from scratch, start by preheating your oven to 350°F (175°C) and greasing and flouring two 9-inch round cake pans. In a large mixing bowl, cream together softened butter and granulated sugar until light and fluffy. Beat in eggs one at a time, making sure each egg is fully incorporated before adding the next. In a separate bowl, sift together all-purpose flour, cocoa powder, baking powder, baking soda, and salt. Divide the batter evenly between the prepared cake pans and bake for 25-30 minutes, or until a toothpick inserted into the center comes out clean.",
]

reference = [
    "The process of making spaghetti carbonara involves boiling pasta, crisping pancetta or guanciale, whisking together eggs and Parmesan cheese, and tossing everything together to create a creamy sauce.",
    "Preparing risotto entails sautéing onions and garlic, toasting Arborio rice, adding wine and broth gradually, and stirring until creamy and tender.",
    "Grilling a flavorful steak involves seasoning generously, preheating the grill, cooking to desired doneness, and letting it rest before slicing.",
    "Creating homemade tomato soup includes sautéing onions and garlic, simmering with tomatoes and broth, pureeing until smooth, and seasoning to taste.",
    "Baking a decadent chocolate cake requires creaming butter and sugar, beating in eggs and alternating dry ingredients with buttermilk before baking until done.",
]

eval_dataset = pd.DataFrame(
    {
        "context": context,
        "reference": reference,
        "instruction": [instruction] * len(context),
    }
)

prompt_templates = [
    "Instruction: {instruction}. Article: {context}. Summary:",
    "Article: {context}. Complete this task: {instruction}, in one sentence. Summary:",
    "Give me a TLDR. Here's an article: {context}. Summary:",
]

generation_config = {
    "temperature": 0.3,
}

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

gemini_model = GenerativeModel(
    "gemini-2.0-flash",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

experiment_name = "eval-sdk-prompt-engineering"

# https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates#overview
metrics = [
    "fluency", #https://cloud.google.com/vertex-ai/generative-ai/docs/models/metrics-templates#pointwise_fluency
    "coherence",
    "groundedness",
    "summarization_quality",
    "verbosity",
]
summarization_eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=metrics,
    experiment=experiment_name,
)


In [2]:
run_id = notebook_utils.generate_uuid(8)
eval_results = []


for i, prompt_template in enumerate(prompt_templates):
    experiment_run_name = f"eval-prompt-engineering-{run_id}-prompt-{i}"

    eval_result = summarization_eval_task.evaluate(
        prompt_template=prompt_template,
        experiment_run_name=experiment_run_name,
        model=gemini_model,
    )

    eval_results.append((f"Prompt #{i}", eval_result))

Associating projects/234439745674/locations/us-central1/metadataStores/default/contexts/eval-sdk-prompt-engineering-eval-prompt-engineering-m8iszh42-prompt-0 to Experiment: eval-sdk-prompt-engineering


Logging Eval Experiment metadata: {'prompt_template': 'Instruction: {instruction}. Article: {context}. Summary:', 'model_name': 'publishers/google/models/gemini-2.0-flash', 'temperature': 0.3, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
Generating a total of 5 responses from Gemini model gemini-2.0-flash.


100%|██████████| 5/5 [00:01<00:00,  2.55it/s]

All 5 responses are successfully generated from Gemini model gemini-2.0-flash.
Multithreaded Batch Inference took: 2.0888872499999707 seconds.
Computing metrics with a total of 25 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 25/25 [00:06<00:00,  3.84it/s]

...

Logging Eval Experiment metadata: {'prompt_template': "Give me a TLDR. Here's an article: {context}. Summary:", 'model_name': 'publishers/google/models/gemini-2.0-flash', 'temperature': 0.3, 'HARM_CATEGORY_UNSPECIFIED': 'BLOCK_NONE', 'HARM_CATEGORY_DANGEROUS_CONTENT': 'BLOCK_NONE', 'HARM_CATEGORY_HATE_SPEECH': 'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT': 'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT': 'BLOCK_NONE'}
Assembling prompts from the `prompt_template`. The `prompt` column in the `EvalResult.metrics_table` has the assembled prompts used for model response generation.
Generating a total of 5 responses from Gemini model gemini-2.0-flash.


100%|██████████| 5/5 [00:01<00:00,  4.58it/s]

All 5 responses are successfully generated from Gemini model gemini-2.0-flash.
Multithreaded Batch Inference took: 1.0968324580026092 seconds.
Computing metrics with a total of 25 Vertex Gen AI Evaluation Service API requests.



100%|██████████| 25/25 [00:06<00:00,  3.64it/s]

All 25 metric requests are successfully computed.
Evaluation Took:6.872241249999206 seconds


In [3]:
for title, eval_result in eval_results:
    notebook_utils.display_eval_result(title=title, eval_result=eval_result)

## Prompt #0

### Summary Metrics

,row_count,fluency/mean,fluency/std,coherence/mean,coherence/std,groundedness/mean,groundedness/std,summarization_quality/mean,summarization_quality/std,verbosity/mean,verbosity/std
0,5.0,5.0,0.0,4.8,0.447214,1.0,0.0,5.0,0.0,0.0,0.0


### Row-based Metrics

,context,reference,instruction,prompt,response,fluency/explanation,fluency/score,coherence/explanation,coherence/score,groundedness/explanation,groundedness/score,summarization_quality/explanation,summarization_quality/score,verbosity/explanation,verbosity/score
0,"To make a classic spaghetti carbonara, start b...",The process of making spaghetti carbonara invo...,Summarize the following article,Instruction: Summarize the following article. ...,"To make spaghetti carbonara, boil salted water...","The response is fluent, with no grammatical er...",5.0,The summary presents a clear and concise versi...,4.0,The response is fully grounded in the provided...,1.0,"The summary is grounded, concise, and fluent, ...",5.0,"The response is perfectly concise, providing a...",0.0
1,Preparing a perfect risotto requires patience ...,Preparing risotto entails sautéing onions and ...,Summarize the following article,Instruction: Summarize the following article. ...,"To make perfect risotto, sauté onions and garl...","The response is completely fluent, free of gra...",5.0,"The summary has a great logical flow, it highl...",5.0,"The response is fully grounded, as it accurate...",1.0,"The summary follows instructions, is grounded,...",5.0,The response is concise and provides all the n...,0.0
2,"For a flavorful grilled steak, start by choosi...",Grilling a flavorful steak involves seasoning ...,Summarize the following article,Instruction: Summarize the following article. ...,"To grill a flavorful steak, choose a marbled c...","The response is fluent, with no grammatical er...",5.0,The response provides a clear and coherent sum...,5.0,The response accurately summarizes the article...,1.0,"The summary follows instructions, is grounded,...",5.0,"The response is appropriately concise, providi...",0.0
3,Creating a creamy homemade tomato soup is a co...,Creating homemade tomato soup includes sautéin...,Summarize the following article,Instruction: Summarize the following article. ...,"To make creamy homemade tomato soup, sauté oni...",The response is completely fluent because it i...,5.0,The response provides a clear and concise summ...,5.0,The summary provided in the response accuratel...,1.0,"The summary follows instructions, is grounded,...",5.0,"The response is appropriately concise, providi...",0.0
4,To bake a decadent chocolate cake from scratch...,Baking a decadent chocolate cake requires crea...,Summarize the following article,Instruction: Summarize the following article. ...,"To bake a chocolate cake, preheat the oven to ...","The response is fluent, with no grammatical er...",5.0,"The summary maintains a logical flow, clear st...",5.0,The response is completely grounded in the pro...,1.0,The response effectively summarizes the provid...,5.0,"The response is perfectly concise, providing a...",0.0


...

## Prompt #2

### Summary Metrics

,row_count,fluency/mean,fluency/std,coherence/mean,coherence/std,groundedness/mean,groundedness/std,summarization_quality/mean,summarization_quality/std,verbosity/mean,verbosity/std
0,5.0,4.6,0.547723,4.4,0.547723,1.0,0.0,5.0,0.0,0.0,0.0


### Row-based Metrics

,context,reference,instruction,prompt,response,fluency/explanation,fluency/score,coherence/explanation,coherence/score,groundedness/explanation,groundedness/score,summarization_quality/explanation,summarization_quality/score,verbosity/explanation,verbosity/score
0,"To make a classic spaghetti carbonara, start b...",The process of making spaghetti carbonara invo...,Summarize the following article,Give me a TLDR. Here's an article: To make a c...,"**TLDR;** Cook pasta, fry pancetta, mix eggs/P...",The response is mostly fluent as it presents a...,4.0,The response is a clear and concise summary of...,4.0,All aspects of the response are attributable t...,1.0,The response follows instructions to provide a...,5.0,The response is appropriately concise and prov...,0.0
1,Preparing a perfect risotto requires patience ...,Preparing risotto entails sautéing onions and ...,Summarize the following article,Give me a TLDR. Here's an article: Preparing a...,**TLDR;** Risotto needs patience! Sauté onions...,The response is fluent and accurately summariz...,5.0,The response presents a coherent summary of th...,4.0,The response only uses information provided in...,1.0,"The response follows the instructions, is grou...",5.0,The response provides a concise summary of the...,0.0
2,"For a flavorful grilled steak, start by choosi...",Grilling a flavorful steak involves seasoning ...,Summarize the following article,Give me a TLDR. Here's an article: For a flavo...,"**TLDR: Get a marbled steak (ribeye/NY strip),...",The response is completely fluent as it provid...,5.0,"The response is completely coherent, providing...",5.0,All information in the TLDR is found within th...,1.0,This response follows instructions by providin...,5.0,"The response is perfectly concise, providing a...",0.0
3,Creating a creamy homemade tomato soup is a co...,Creating homemade tomato soup includes sautéin...,Summarize the following article,Give me a TLDR. Here's an article: Creating a ...,**TLDR;** Make creamy tomato soup by sautéing ...,"The response is concise, grammatically correct...",5.0,"The response has a seamless logical flow, is e...",5.0,The response accurately summarizes the provide...,1.0,The response successfully provides a concise a...,5.0,"The response is perfectly concise, providing a...",0.0
4,To bake a decadent chocolate cake from scratch...,Baking a decadent chocolate cake requires crea...,Summarize the following article,Give me a TLDR. Here's an article: To bake a d...,"TLDR: Preheat oven, grease pans. Cream butter ...",The response is mostly fluent as it has very f...,4.0,The response presents a coherent summary of th...,4.0,All aspects of the response are attributable t...,1.0,The response is a good TLDR; it follows instru...,5.0,The response is concise and provides all the n...,0.0


In [4]:

for title, eval_result in eval_results:
    notebook_utils.display_explanations(eval_result, metrics=["summarization_quality"])

In [5]:
notebook_utils.display_radar_plot(
    eval_results,
    metrics=metrics,
)
notebook_utils.display_bar_plot(
    eval_results,
    metrics=metrics,
)